In [1]:
import numpy as np
import pandas as pd
import sys
import time
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Input, Lambda, Flatten, Dense, Reshape, Add, Dropout, LSTM
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
import pickle

Using TensorFlow backend.


### Image sample

In [11]:
img = image.load_img('jpegs_256/ApplyEyeMakeup_0/frame000001.jpg', target_size=(224,224))
img = image.img_to_array(img)

In [12]:
max_frame = 0
min_frame = 20000
for folder in os.listdir('jpegs_256/'):
    frame = len(os.listdir('jpegs_256/'+folder+'/'))
    if frame > max_frame:
        max_frame = frame
        name1 = folder
    if frame < min_frame:
        min_frame = frame
        name2 = folder
max_frame, name1, min_frame, name2

KeyboardInterrupt: 

In [4]:
num_categories, folder_names, Xtrain, ytrain, Xtest, ytest = pickle.load(open('Data.p', 'rb'))

In [5]:
num_categories

{'ApplyEyeMakeup': 145,
 'ApplyLipstick': 114,
 'Archery': 145,
 'BabyCrawling': 132,
 'BalanceBeam': 108,
 'BandMarching': 155,
 'BaseballPitch': 150,
 'BasketballDunk': 131,
 'Basketball': 134,
 'BenchPress': 160,
 'Biking': 134,
 'Billiards': 150,
 'BlowDryHair': 131,
 'BlowingCandles': 109,
 'BodyWeightSquats': 112,
 'Bowling': 155,
 'BoxingPunchingBag': 163,
 'BoxingSpeedBag': 134,
 'BreastStroke': 101,
 'BrushingTeeth': 131,
 'CleanAndJerk': 112,
 'CliffDiving': 138,
 'CricketBowling': 139,
 'CricketShot': 167,
 'CuttingInKitchen': 110,
 'Diving': 150,
 'Drumming': 161,
 'Fencing': 111,
 'FieldHockeyPenalty': 126,
 'FloorGymnastics': 125,
 'FrisbeeCatch': 126,
 'FrontCrawl': 137,
 'GolfSwing': 139,
 'Haircut': 130,
 'Hammering': 140,
 'HammerThrow': 150,
 'HandstandPushups': 128,
 'HandstandWalking': 111,
 'HeadMassage': 147,
 'HighJump': 123,
 'HorseRace': 124,
 'HorseRiding': 164,
 'HulaHoop': 125,
 'IceDancing': 158,
 'JavelinThrow': 117,
 'JugglingBalls': 121,
 'JumpingJack':

### Feature Selection using VGG19

In [6]:
VGG = VGG19(include_top = False, weights = 'imagenet', input_tensor = Input((224,224, 3)), input_shape = (224,224,3), pooling = 'max')

W0215 16:36:14.432578  4144 deprecation_wrapper.py:119] From C:\Users\Acer\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0215 16:36:14.482444  4144 deprecation_wrapper.py:119] From C:\Users\Acer\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0215 16:36:14.490423  4144 deprecation_wrapper.py:119] From C:\Users\Acer\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0215 16:36:14.526326  4144 deprecation_wrapper.py:119] From C:\Users\Acer\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0215 16:36:15.249695  4144 deprecation_wrapper.py:119] From C:\Users\Acer\Anaconda

In [7]:
VGG.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [113]:
start = time.time()
for folder in os.listdir('jpegs_256/')[6803:]:
    print(folder)
    batch = []
    for img_name in os.listdir('jpegs_256/' + folder + '/'):
        img = image.load_img('jpegs_256/'+folder+'/'+img_name, target_size=(224,224))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        batch.append(img)
    batch = VGG.predict(np.array(batch))
    end = time.time()
    np.save('NumPy/Features_'+folder.split('_')[1]+'.npy', batch)
    print(batch.shape, end-start)

LongJump_10099
(145, 512) 3.0757319927215576
LongJump_10100
(92, 512) 4.485956907272339
LongJump_10101
(150, 512) 7.630570411682129
LongJump_10102
(191, 512) 10.328325510025024
LongJump_10103
(114, 512) 12.018802404403687
LongJump_10104
(150, 512) 14.169050455093384
LongJump_10105
(150, 512) 16.475903034210205
LongJump_10106
(150, 512) 18.69298243522644
LongJump_10107
(166, 512) 21.246116399765015
LongJump_10108
(197, 512) 24.141366958618164
LongJump_10109
(200, 512) 27.15629816055298
LongJump_10110
(159, 512) 29.441189289093018
LongJump_10111
(118, 512) 31.348084211349487
LongJump_10112
(115, 512) 33.07945275306702
LongJump_10113
(118, 512) 34.88763165473938
LongJump_10114
(148, 512) 37.122633934020996
LongJump_10115
(141, 512) 39.25592398643494
LongJump_10116
(106, 512) 40.84766721725464
LongJump_10117
(122, 512) 42.74558758735657
LongJump_10118
(165, 512) 45.28184247016907
LongJump_10119
(142, 512) 47.475931882858276
LongJump_10120
(155, 512) 49.88847494125366
LongJump_10121
(149, 5

(144, 512) 582.2466104030609
MilitaryParade_10429
(143, 512) 584.63219165802
MilitaryParade_10430
(156, 512) 587.1165792942047
MilitaryParade_10431
(148, 512) 589.50119972229
MilitaryParade_10432
(140, 512) 591.8478832244873
MilitaryParade_10433
(126, 512) 593.8734655380249
MilitaryParade_10434
(106, 512) 595.5888752937317
MilitaryParade_10435
(143, 512) 597.9914462566376
MilitaryParade_10436
(122, 512) 600.1207509040833
MilitaryParade_10437
(78, 512) 601.3834054470062
MilitaryParade_10438
(99, 512) 603.098824262619
MilitaryParade_10439
(123, 512) 605.1383247375488
MilitaryParade_10440
(97, 512) 606.7549982070923
MilitaryParade_10441
(105, 512) 608.4743971824646
MilitaryParade_10442
(109, 512) 610.2666015625
MilitaryParade_10443
(135, 512) 612.5455043315887
MilitaryParade_10444
(126, 512) 614.6598484516144
MilitaryParade_10445
(136, 512) 616.896862745285
MilitaryParade_10446
(102, 512) 618.719982624054
MilitaryParade_10447
(122, 512) 620.690710067749
MilitaryParade_10448
(89, 512) 622.

(104, 512) 959.8106782436371
Mixing_10682
(99, 512) 961.4383616447449
Mixing_10683
(91, 512) 963.0380425453186
Mixing_10684
(116, 512) 965.058965921402
Mixing_10685
(116, 512) 966.9798672199249
Mixing_10686
(103, 512) 969.2986283302307
Mixing_10687
(113, 512) 971.7291181087494
Mixing_10688
(103, 512) 973.4744477272034
Mixing_10689
(121, 512) 975.6077768802643
Mixing_10690
(102, 512) 977.2463936805725
Mixing_10691
(102, 512) 978.9647579193115
Mixing_10692
(98, 512) 980.5086266994476
Mixing_10693
(96, 512) 982.2509658336639
Mixing_10694
(103, 512) 984.0421726703644
Mixing_10695
(91, 512) 985.5381696224213
Mixing_10696
(104, 512) 987.2356305122375
Mixing_10697
(107, 512) 989.1026315689087
Mixing_10698
(110, 512) 990.3832070827484
Mixing_10699
(116, 512) 992.3280420303345
Mixing_10700
(92, 512) 993.8549163341522
Mixing_10701
(114, 512) 995.9074277877808
Mixing_10702
(114, 512) 997.8472344875336
Mixing_10703
(81, 512) 999.2265472412109
Mixing_10704
(101, 512) 1000.9918200969696
Mixing_10705

(251, 512) 1489.7913279533386
Nunchucks_11012
(251, 512) 1494.2523906230927
Nunchucks_11013
(250, 512) 1498.5678446292877
Nunchucks_11014
(252, 512) 1501.6336417198181
Nunchucks_11015
(251, 512) 1506.0144937038422
Nunchucks_11016
(244, 512) 1510.5274186134338
Nunchucks_11017
(261, 512) 1514.856190443039
Nunchucks_11018
(261, 512) 1519.1118848323822
Nunchucks_11019
(258, 512) 1523.4128377437592
Nunchucks_11020
(258, 512) 1527.6362385749817
Nunchucks_11021
(268, 512) 1532.1564373970032
Nunchucks_11022
(251, 512) 1536.33039188385
Nunchucks_11023
(226, 512) 1540.1376729011536
Nunchucks_11024
(209, 512) 1543.6394381523132
Nunchucks_11025
(205, 512) 1547.1176195144653
Nunchucks_11026
(216, 512) 1550.7233860492706
Nunchucks_11027
(318, 512) 1556.0646986961365
Nunchucks_11028
(220, 512) 1560.2802169322968
Nunchucks_11029
(210, 512) 1563.8467404842377
Nunchucks_11030
(212, 512) 1567.3563742637634
Nunchucks_11031
(255, 512) 1571.66273021698
Nunchucks_11032
(255, 512) 1576.0648696422577
Nunchucks

(144, 512) 2131.918251514435
ParallelBars_11256
(125, 512) 2134.021624803543
ParallelBars_11257
(101, 512) 2135.807844877243
ParallelBars_11258
(198, 512) 2139.3004989624023
ParallelBars_11259
(143, 512) 2141.826740503311
ParallelBars_11260
(166, 512) 2144.7369525432587
ParallelBars_11261
(151, 512) 2147.3170490264893
ParallelBars_11262
(159, 512) 2150.128526210785
ParallelBars_11263
(185, 512) 2153.3010382652283
ParallelBars_11264
(141, 512) 2155.707602262497
ParallelBars_11265
(167, 512) 2158.583902359009
ParallelBars_11266
(120, 512) 2160.773045539856
ParallelBars_11267
(137, 512) 2163.350150346756
ParallelBars_11268
(152, 512) 2166.1207377910614
ParallelBars_11269
(108, 512) 2168.095494747162
ParallelBars_11270
(165, 512) 2171.092433691025
ParallelBars_11271
(145, 512) 2173.8181760311127
ParallelBars_11272
(121, 512) 2176.041193008423
ParallelBars_11273
(157, 512) 2178.743703365326
ParallelBars_11274
(118, 512) 2180.8999724388123
ParallelBars_11275
(143, 512) 2183.4810287952423
Par

...

(116, 512) 22292.734748840332
UnevenBars_19049
(166, 512) 22295.750927448273
UnevenBars_19050
(173, 512) 22298.861166000366
UnevenBars_19051
(139, 512) 22301.07031393051
UnevenBars_19052
(121, 512) 22303.373639583588
UnevenBars_19053
(171, 512) 22306.664373636246
UnevenBars_19054
(185, 512) 22309.98769044876
UnevenBars_19055
(109, 512) 22311.98623895645
UnevenBars_19056
(63, 512) 22313.113149642944
UnevenBars_19057
(122, 512) 22315.372804641724
UnevenBars_19058
(198, 512) 22318.95267534256
UnevenBars_19059
(257, 512) 22323.68483352661
UnevenBars_19060
(163, 512) 22326.642788887024
UnevenBars_19061
(168, 512) 22329.781491279602
UnevenBars_19062
(146, 512) 22332.49387192726
UnevenBars_19063
(314, 512) 22338.207357883453
UnevenBars_19064
(302, 512) 22343.674082756042
UnevenBars_19065
(299, 512) 22349.222462892532
UnevenBars_19066
(299, 512) 22354.51796603203
UnevenBars_19067
(325, 512) 22360.40105319023
UnevenBars_19068
(127, 512) 22364.335726499557
UnevenBars_19069
(132, 512) 22368.35967

(118, 512) 22670.14390015602
VolleyballSpiking_19306
(203, 512) 22673.51437163353
VolleyballSpiking_19307
(261, 512) 22678.001305103302
VolleyballSpiking_19308
(117, 512) 22680.275038957596
VolleyballSpiking_19309
(101, 512) 22682.096985578537
VolleyballSpiking_19310
(121, 512) 22684.201634645462
VolleyballSpiking_19311
(133, 512) 22687.019203424454
VolleyballSpiking_19312
(76, 512) 22688.332802057266
VolleyballSpiking_19313
(101, 512) 22690.150762319565
VolleyballSpiking_19314
(179, 512) 22694.2229681015
VolleyballSpiking_19315
(102, 512) 22696.029459238052
WalkingWithDog_19400
(240, 512) 22700.30988430977
WalkingWithDog_19401
(240, 512) 22705.56061911583
WalkingWithDog_19402
(240, 512) 22709.927018642426
WalkingWithDog_19403
(180, 512) 22713.42266535759
WalkingWithDog_19404
(240, 512) 22718.04729437828
WalkingWithDog_19405
(240, 512) 22723.763001918793
WalkingWithDog_19406
(240, 512) 22728.4162838459
WalkingWithDog_19407
(240, 512) 22732.904237270355
WalkingWithDog_19408
(240, 512) 2

(94, 512) 23296.906136274338
WallPushups_19631
(94, 512) 23298.647477149963
WallPushups_19632
(101, 512) 23300.612218379974
WallPushups_19633
(84, 512) 23302.361577510834
WallPushups_19634
(84, 512) 23303.879811763763
WallPushups_19635
(139, 512) 23306.35419034958
WallPushups_19636
(150, 512) 23309.043992996216
WallPushups_19637
(152, 512) 23311.88339662552
WallPushups_19638
(120, 512) 23314.21316218376
WallPushups_19639
(129, 512) 23316.326509475708
WallPushups_19640
(75, 512) 23317.78460597992
WallPushups_19641
(86, 512) 23319.393302202225
WallPushups_19642
(112, 512) 23321.60538291931
WallPushups_19643
(103, 512) 23323.647917985916
WallPushups_19644
(124, 512) 23325.955743789673
WallPushups_19645
(92, 512) 23327.578401088715
WallPushups_19646
(69, 512) 23328.866953611374
WallPushups_19647
(71, 512) 23330.26820421219
WallPushups_19648
(79, 512) 23331.56673002243
WallPushups_19649
(62, 512) 23332.819378376007
WallPushups_19650
(154, 512) 23335.685708761215
WallPushups_19651
(162, 512)

(101, 512) 23690.34728860855
WritingOnBoard_19870
(101, 512) 23693.42634677887
WritingOnBoard_19871
(169, 512) 23698.3514213562
WritingOnBoard_19872
(101, 512) 23700.16343855858
WritingOnBoard_19873
(124, 512) 23702.328411340714
WritingOnBoard_19874
(111, 512) 23704.281074285507
WritingOnBoard_19875
(205, 512) 23707.835263490677
WritingOnBoard_19876
(144, 512) 23710.367704629898
WritingOnBoard_19877
(101, 512) 23712.23701119423
WritingOnBoard_19878
(203, 512) 23715.77105665207
WritingOnBoard_19879
(161, 512) 23718.58585357666
WritingOnBoard_19880
(222, 512) 23722.501513004303
WritingOnBoard_19881
(158, 512) 23725.352477788925
WritingOnBoard_19882
(137, 512) 23728.048154354095
WritingOnBoard_19883
(99, 512) 23730.03991651535
WritingOnBoard_19884
(86, 512) 23731.630350112915
WritingOnBoard_19885
(205, 512) 23735.250644922256
WritingOnBoard_19886
(227, 512) 23739.4455370903
WritingOnBoard_19887
(193, 512) 23742.916110754013
WritingOnBoard_19888
(267, 512) 23747.6992251873
WritingOnBoard_1

(168, 512) 24295.705694913864
YoYo_20100
(186, 512) 24299.005308389664
YoYo_20101
(185, 512) 24302.334295988083
YoYo_20102
(194, 512) 24305.999371290207
YoYo_20103
(178, 512) 24311.35775732994
YoYo_20104
(177, 512) 24314.67288708687
YoYo_20105
(167, 512) 24317.71178483963
YoYo_20106
(183, 512) 24321.09270977974
YoYo_20107
(191, 512) 24324.822733163834
YoYo_20108
(190, 512) 24328.512856006622
YoYo_20109
(182, 512) 24332.038422346115
YoYo_20110
(198, 512) 24335.71152997017
YoYo_20111
(212, 512) 24339.711824417114
YoYo_20112
(199, 512) 24343.71008992195
YoYo_20113
(205, 512) 24348.76759338379
YoYo_20114
(172, 512) 24351.942101716995
YoYo_20115
(165, 512) 24354.879204511642
YoYo_20116
(212, 512) 24358.62717437744
YoYo_20117
(218, 512) 24362.56467962265
YoYo_20118
(185, 512) 24366.022430419922
YoYo_20119
(193, 512) 24369.813245534897
YoYo_20120
(193, 512) 24373.38269519806
YoYo_20121
(195, 512) 24376.898327589035
YoYo_20122
(194, 512) 24380.461753606796
YoYo_20123
(175, 512) 24383.946428775

In [10]:
folder = 'jpegs_256/LongJump_10098/'
batch = []
for img_name in os.listdir(folder):
    img = image.load_img(folder+img_name, target_size=(224,224))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    batch.append(img)
batch = VGG.predict(np.array(batch))
end = time.time()
np.save('Features_10098.npy', batch)